# En ny learning och validering   

Beskrivning:  

Normal learning:  
- Learning: gör webscrape av omgång(ar) och kör en learning

GridSearch optimering:  
- gör gridsearch av de olika modellerna typ1, typ6, typ9 och typ16 samt meta_modellen
  - inkludera imbalanced-lösningar
  - spara de bästa hyperparametrarna
  
Värdera:  
- Cross_validate: ts_split and ts cross_validate of the stack of models plus the meta_model
  - save the results in pickle
- Vinst: beräknar vinsten för de olika modellerna

### Först kolla artiklar om stacking, cv för stacking samt cv för stacking av timeseries 
- https://machinelearningmastery.com/implementing-stacking-scratch-python/   (Även kod i Pieces)  
Även allmänt om stacking ensembles  
- https://machinelearningmastery.com/essence-of-stacking-ensembles-for-machine-learning/  
Slutligen CV för Timeseries stacking  (se kod i Pieces)  
- https://datascience.stackexchange.com/questions/41378/how-to-apply-stacking-cross-validation-for-time-series-data

# Generella funktioner

In [1]:
# moduler
from sklearn.ensemble import RandomForestRegressor
import sys
import pandas as pd
import numpy as np
import pickle
from catboost import CatBoostClassifier, Pool, cv
from IPython.display import display
pd.set_option('display.width', 200)
pd.set_option('display.max_columns', 200)

sys.path.append(
    'C:\\Users\\peter\\Documents\\MyProjects\\PyProj\\Trav\\spel\\')

import typ as tp


In [2]:

# läs in data
def läs_in_data_och_förbered():
    df = pd.read_csv('..\\all_data.csv')
    # Följande datum saknar avd==5 och kan inte användas
    saknas = ['2015-08-15', '2016-08-13', '2017-08-12']
    df = df[~df.datum.isin(saknas)]
    X = df.copy()
    X.drop('plac', axis=1, inplace=True)
    
    y = (df.plac == 1)*1   # plac 1 eller 0

    for f in ['häst', 'bana', 'kusk', 'h1_kusk', 'h2_kusk', 'h3_kusk', 'h4_kusk', 'h5_kusk', 'h1_bana', 'h2_bana', 'h3_bana', 'h4_bana', 'h5_bana']:
        X[f] = X[f].str.lower()

    X.reset_index(drop=True, inplace=True)
    y.reset_index(drop=True, inplace=True)
    
    # läs in FEATIRES.txt
    with open('..\\FEATURES.txt', 'r') as f:    
        features = f.read().splitlines()
    
    assert len(features) == len(X.columns), f'features {len(features)} and X.columns {len(X.columns)} are not the same length'   
    assert set(features) == set(X.columns), f'features {set(features)} and X.columns {set(X.columns)} are not the same'
    
    return X, y


In [3]:
#              namn, ant_hästar, proba, Kelly,  motst_ant, motst_diff, ant_favoriter,   only_clear, streck
typ6 = tp.Typ('typ6', True,       True, False,     0,       False,          0,            False,    True)
typ1 = tp.Typ('typ1', False,      True, False,     2,       True,           2,            True,     False)
typ9 = tp.Typ('typ9', True,       True, True,      2,       True,           2,            True,     True)
typ16= tp.Typ('typ16',True,       True, True,      2,       True,           2,            False,    True)

typer = [typ6, typ1, typ9, typ16]


In [4]:
def remove_features(df_, remove_mer=[]):
    df = df_.copy()
    df.drop(['startnr', 'vodds', 'podds', 'bins', 'h1_dat',
             'h2_dat', 'h3_dat', 'h4_dat', 'h5_dat'], axis=1, inplace=True)
    if remove_mer:
        df.drop(remove_mer, axis=1, inplace=True)

    return df

# GridSearchCV av typ6, typ1, typ9, typ16

# DATA
    # Läs in data
    # remove_features

# modeller konfigurering, CatBoostClassifier  
    

In [5]:

# create a GridSearch for CatBoostClassifier
from sklearn.model_selection import TimeSeriesSplit
def create_grid_search(X_, y_, typ, verbose=False):
    X = X_.copy()
    y = y_.copy()
    # remove features
    X = typ.prepare_for_model(X)
    if not typ.streck:
        X.drop('streck', axis=1, inplace=True)
    
    X, cat_features = tp.prepare_for_catboost(X)
    print('cat_features\n',cat_features)
    X = remove_features(X, remove_mer=['datum','avd'])
    # get numerical features and cat_features
    num_features = list(X.select_dtypes(include=[np.number]).columns)
    cat_features = list(X.select_dtypes(include=['object']).columns)
    # print('cat_features:\n',cat_features,'\n')
    # print(X[cat_features].info())
    assert X[cat_features].isnull().sum().sum() == 0, 'there are NaN values in cat_features'
    # create a GridSearch
    param_grid = {
        'learning_rate': [0.1, 0.01, 0.001, 0.0001],
        'depth': [3, 4, 5, 6, 7, 8, 9, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9, 11, 13, 15],
        'iterations': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
        'bagging_temperature': [0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 1.9],
        'rsm': [0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 1.9],
        'depth_per_tree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        'border_count': [32, 64, 128, 256, 512, 1024, 2048, 4096],
        'fold_permutation_block_size': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        'od_pval': [0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 1.9],
        'od_wait': [0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3,  1.5, 1.7, 1.9],
        'od_type': ['IncToDec', 'Iter'],
    }

    # X.datum = pd.to_datetime(X.datum)
    # X.datum = X.datum.dt.date
    # alla_datum = X.datum.unique()

    tscv = TimeSeriesSplit(n_splits=5)

    # for train_index, test_index in tscv.split(X):
    #     print("TRAIN:", train_index, "TEST:", test_index)
        # X_train, X_test = X[train_index], X[test_index]
        # y_train, y_test = y[train_index], y[test_index]

    model = CatBoostClassifier(iterations=500, loss_function='Logloss', eval_metric='AUC',
                               use_best_model=False, early_stopping_rounds=100, verbose=verbose,)

    grid = {'learning_rate': [0.002,0.005],
            'depth': [2,4,6,8],
            'l2_leaf_reg': [8, 9, 10,11]}
    
    # print(X[cat_features].info())
    assert X[cat_features].isnull().sum().sum() == 0, 'there are NaN values in cat_features'

    grid_search_result = model.grid_search(grid,
                                        X=Pool(X, y, cat_features=cat_features),
                                        cv=tscv.split(X),
                                        shuffle=False,
                                        search_by_train_test_split=False,
                                        verbose=verbose,
                                        plot=True)
    return grid_search_result


## Testa grid_search

In [6]:
# Total GridSearchCV där jag använder egen TimeseriesSplit
if False:
    X, y = läs_in_data_och_förbered()
    results6 = create_grid_search(X, y, typ6)

    X, y = läs_in_data_och_förbered()
    results1 = create_grid_search(X, y, typ1)

    X,y = läs_in_data_och_förbered()
    results9 = create_grid_search(X, y, typ9)

    X, y = läs_in_data_och_förbered()
    results16 = create_grid_search(X, y, typ16)


In [7]:
dfResults = pd.read_csv('results.csv')
def results(typ, dfResults, comm=[]):
    s = "results"+str(typ)
    temp1 =globals()[s]
    temp = pd.DataFrame(temp1['cv_results'])
    temp['typ'] = typ
    max=temp['test-AUC-mean'].max()
    temp=temp[temp['test-AUC-mean']==max]
    if len(comm)>0:
        temp['comm'] = comm
    str(temp1['params'])
    temp['best_params'] = str(temp1['params']).replace('{','').replace('}','').replace('depth','d').replace('l2_leaf_reg','l2').replace('learning_rate','LR')
    dfResults = pd.concat([dfResults,temp])
    return dfResults

#### Slår ihop alla resultaten från GridSearch i funktionen ovan
if False:
    dfResults = results(1, dfResults,  'ökade iter=500')
    dfResults = results(6, dfResults,  'ökade iter=500')
    dfResults = results(9, dfResults,  'ökade iter=500')
    dfResults = results(16, dfResults, 'ökade iter=500')
    dfResults.to_csv('results.csv', index=False)
dfResults


,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std,typ,best_params,comm
0,296,0.797722,0.003251,0.263087,0.002299,0.260534,0.000938,1,"'d': 4, 'l2': 9, 'LR': 0.005",NaN
1,283,0.813249,0.003424,0.259381,0.001801,0.255515,0.001394,6,"'d': 6, 'l2': 9, 'LR': 0.005",NaN
2,297,0.813179,0.004967,0.257471,0.001976,0.254232,0.000886,9,"'d': 4, 'l2': 10, 'LR': 0.005",NaN
3,297,0.813179,0.004967,0.257471,0.001976,0.254232,0.000886,16,"'d': 4, 'l2': 10, 'LR': 0.005",NaN
4,298,0.798138,0.004514,0.267621,0.011353,0.265052,0.011595,1,"'d': 4, 'l2': 11, 'LR': 0.005",fler val
5,299,0.813913,0.004760,0.258810,0.001824,0.256468,0.000602,6,"'d': 2, 'l2': 9, 'LR': 0.005",fler val
6,297,0.813179,0.004967,0.257471,0.001976,0.254232,0.000886,9,"'d': 4, 'l2': 10, 'LR': 0.005",fler val
7,297,0.813179,0.004967,0.257471,0.001976,0.254232,0.000886,16,"'d': 4, 'l2': 10, 'LR': 0.005",fler val
8,432,0.799307,0.003276,0.260090,0.021130,0.256483,0.024002,1,"'d': 4, 'l2': 8, 'LR': 0.005",ökade iter=500
9,460,0.815003,0.004842,0.244052,0.002184,0.240895,0.001442,6,"'d': 2, 'l2': 9, 'LR': 0.005",ökade iter=500


- Spara modeller och optimala parms (kanske skall läras på del 1-4?)
- Predict på den sista 5:e delen och skapa input till meta-model
- gridsearchcv meta_model på detta. Spara optimala parms
- avsluta med en learn på hela 5:- delen med optimala parms och spara

In [8]:
def learn_temp(typ, X_, y_, iterations, depth, learning_rate, l2_leaf_reg,save=False):
    X = X_.copy()
    y = y_.copy()
    
    typ.learn(X, y, learning_rate=learning_rate, iterations=iterations, depth=depth, l2_leaf_reg=l2_leaf_reg, save=save)


In [9]:
from sklearn.model_selection import TimeSeriesSplit
def learn_all_models(totals = False, save=False):
    tscv = TimeSeriesSplit(n_splits=5)
    X, y = läs_in_data_och_förbered()
    for train_index, test_index in tscv.split(X):
        #print("TRAIN:", train_index[-1:], "TEST:", test_index[0])
        train_until = train_index[-1:][0]
        test_from = train_until+1

    if totals == False:
        ### sista 5:e delen av data
        print('train_until',train_until, 'test_from',test_from)

        X_train, X_test = X.iloc[:test_from], X.iloc[test_from:]
        y_train, y_test = y[:test_from], y[test_from:]
    else:
        ### all data
        print('train all')

        X_train = X.copy()
        y_train = y.copy()
    results = pd.read_csv('results.csv')

    # typ6: 'd': 2, 'l2': 9, 'LR': 0.005
    print(results[results.typ==6][['typ','iterations','best_params']].iloc[-1].values)
    iter=460
    d=2
    LR=0.005
    L2=9
    learn_temp(typ6, X_train, y_train, iterations=iter, depth=d, learning_rate=LR, l2_leaf_reg=L2,save=save)

    # typ1: 'd': 4, 'l2': 8, 'LR': 0.005
    print(results[results.typ == 1][['typ', 'iterations', 'best_params']].iloc[-1].values)
    iter = 432
    d = 4
    LR = 0.005
    L2 = 9
    learn_temp(typ1, X_train, y_train, iterations=iter, depth=d,
               learning_rate=LR, l2_leaf_reg=L2, save=save)
    # typ9: 'd': 2, 'l2': 10, 'LR': 0.005
    print(results[results.typ == 9][['typ', 'iterations', 'best_params']].iloc[-1].values)
    iter = 496
    d = 2
    LR = 0.005
    L2 = 10
    learn_temp(typ9, X_train, y_train, iterations=iter, depth=d,
               learning_rate=LR, l2_leaf_reg=L2, save=save)

    # typ16:'d': 2, 'l2': 10, 'LR': 0.005
    print(results[results.typ == 16][['typ', 'iterations', 'best_params']].iloc[-1].values)
    iter = 496
    d = 2
    LR = 0.005
    L2 = 10
    learn_temp(typ16, X_train, y_train, iterations=iter, depth=d,
               learning_rate=LR, l2_leaf_reg=L2, save=save)

if False:
    learn_all_models()


## Learn meta_model

In [10]:
# Skapa ett Kelly-värde baserat på streck omvandlat till odds
def kelly(proba, streck, odds):  # proba = prob winning, streck i % = streck
    with open('rf_streck_odds.pkl', 'rb') as f:
        rf = pickle.load(f)

    if odds is None:
        o = rf.predict(streck.copy())
    else:
        o = rf.predict(streck.copy())

    # for each values > 40 in odds set to 1
    o[o > 40] = 1
    return (o*proba - (1-proba))/o


In [11]:
# stack for learning meta_model
def skapa_stack_learning(X_, y, save=True):
    X=X_.copy()
    stacked_data = pd.DataFrame()
    for typ in typer:
            nr = typ.name[3:]
            stacked_data['proba'+nr] = typ.predict(X)
            stacked_data['kelly' + nr] = kelly(stacked_data['proba' + nr], X[['streck']], None)

    print(stacked_data.columns)
    assert len(stacked_data) == len(y), f'stacked_data {len(stacked_data)} and y {len(y)} should have same length'
    return stacked_data,y   # enbart stack-info

if True: # förbered data och kör stacking
    tscv = TimeSeriesSplit(n_splits=5)
    X, y = läs_in_data_och_förbered()
    for train_index, test_index in tscv.split(X):
        #print("TRAIN:", train_index[-1:], "TEST:", test_index[0])
        train_from = test_index[0]   # sic!

    ### sista 5:e delen av data
    print('train_from for meta', train_from)

    X_train, y_train = X.iloc[train_from:], y[train_from:]
    X_stack,y_stack = skapa_stack_learning(X_train, y_train, False)
    display(X_stack)


train_from for meta 36554
typ6, Lägg in ant_hästar, 
Number of NaN in cat before: 10
Number of NaN in cat after: 0
predict typ6
typ1, Lägg in diff motståndare, 
Number of NaN in cat before: 10
Number of NaN in cat after: 0
predict typ1
typ9, Lägg in ant_hästar, Lägg in diff motståndare, 
Number of NaN in cat before: 10
Number of NaN in cat after: 0
predict typ9
typ16, Lägg in ant_hästar, Lägg in diff motståndare, 
Number of NaN in cat before: 10
Number of NaN in cat after: 0
predict typ16
Index(['proba6', 'kelly6', 'proba1', 'kelly1', 'proba9', 'kelly9', 'proba16', 'kelly16'], dtype='object')


,proba6,kelly6,proba1,kelly1,proba9,kelly9,proba16,kelly16
0,0.056455,-0.026782,0.043391,-0.040998,0.063922,-0.018656,0.063922,-0.018656
1,0.294226,0.105674,0.301050,0.114322,0.248929,0.048276,0.248929,0.048276
2,0.004607,-0.026405,0.004855,-0.026150,0.014200,-0.016514,0.014200,-0.016514
3,0.296817,0.063331,0.252327,0.004068,0.388570,0.185549,0.388570,0.185549
4,0.107029,-0.085896,0.105438,-0.087832,0.109900,-0.082405,0.109900,-0.082405
...,...,...,...,...,...,...,...,...
7305,0.031760,-0.003367,0.012636,-0.023185,0.024499,-0.010891,0.024499,-0.010891
7306,0.012385,-0.018385,0.033464,0.003350,0.037044,0.007042,0.037044,0.007042
7307,0.067805,0.014609,0.050618,-0.003559,0.095576,0.043964,0.095576,0.043964
7308,0.277928,0.121925,0.328757,0.183736,0.196334,0.022702,0.196334,0.022702


In [12]:
# fit meta_model
def learn_meta_model(X, y,  save=True,
                     n_estimators=100, 
                     max_depth=None,
                     min_samples_split=2, 
                     min_samples_leaf=1, 
                     max_features='auto', 
                     max_leaf_nodes=None,
                     max_samples=None,):
    from sklearn.ensemble import RandomForestClassifier

    print('\nFitting meta_model on X with all models predictions')

    meta_model = RandomForestClassifier(
       class_weight='balanced',    #{0:1,1:10},
       oob_score=True, verbose=1, n_jobs=8, random_state=2022,
        max_depth=max_depth,
        n_estimators=n_estimators, 
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        max_leaf_nodes=max_leaf_nodes,
        max_samples=max_samples,)
    
    meta_model.fit(X, y)

    print('OOB_score', meta_model.oob_score_)   # 0.9430916552667579
    # pickle save stacking
    if save:
        with open('../modeller/meta_model.model', 'wb') as f:
            pickle.dump(meta_model, f)

    return meta_model

if True:
    grid ={
        'n_estimators': [10,100,200],
        'max_depth': [None,2,4,6],
        'min_samples_split': [2,4,6],
        'min_samples_leaf':[1],  # 2,3],
        'max_features': ['auto'], # 'log2', None],
        'max_leaf_nodes': [None], # 20, 10, 5],
        'max_samples':[None], # 0.1, 0.2, 0.4, 0.6, 0.8, 1.0],
    }
    
    oob_dict = {}
    n=1
    keys = list(grid.keys())
    for i, key in enumerate(keys[:3]):
        # loop over all values in key
        for value in grid[key]:
            for key2 in keys[i+1:3]:
                for value2 in grid[key2]:
                    for key3 in keys[i+2:3]:
                        for value3 in grid[key3]:
                            
                            print('konf', n, key, value, key2, value2, key3, value3)
                
                            meta_model = learn_meta_model(X_stack, y_stack, save=True,
                                                  n_estimators = value,
                                                  max_depth = value2,
                                                  min_samples_split = value3,
                                                  min_samples_leaf = 1,  # 2,3], 
                                                  max_features = 'auto', # 'log2', None], 
                                                  max_leaf_nodes = None, # 20, 10, 5],  
                                                  max_samples    = None,)
                            # keep n:oob_score in dict
                            oob_dict[n] = meta_model.oob_score_
        
                            n+=1
    print(oob_dict)


konf 1 n_estimators 10 max_depth None min_samples_split 2

Fitting meta_model on X with all models predictions
OOB_score 0.9320109439124487
konf 2 n_estimators 10 max_depth None min_samples_split 4

Fitting meta_model on X with all models predictions
OOB_score 0.9292749658002736
konf 3 n_estimators 10 max_depth None min_samples_split 6

Fitting meta_model on X with all models predictions
OOB_score 0.9264021887824897
konf 4 n_estimators 10 max_depth 2 min_samples_split 2

Fitting meta_model on X with all models predictions


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
C:\Users\peter\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\peter\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  decision = (predictions[k] /
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
C:\Users\peter\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB score

OOB_score 0.8177838577291382
konf 5 n_estimators 10 max_depth 2 min_samples_split 4

Fitting meta_model on X with all models predictions
OOB_score 0.8177838577291382
konf 6 n_estimators 10 max_depth 2 min_samples_split 6

Fitting meta_model on X with all models predictions
OOB_score 0.8177838577291382
konf 7 n_estimators 10 max_depth 4 min_samples_split 2

Fitting meta_model on X with all models predictions
OOB_score 0.8341997264021888
konf 8 n_estimators 10 max_depth 4 min_samples_split 4

Fitting meta_model on X with all models predictions
OOB_score 0.8341997264021888
konf 9 n_estimators 10 max_depth 4 min_samples_split 6

Fitting meta_model on X with all models predictions


[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
C:\Users\peter\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\peter\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  decision = (predictions[k] /
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
C:\Users\peter\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\peter\anaconda3\lib

OOB_score 0.8341997264021888
konf 10 n_estimators 10 max_depth 6 min_samples_split 2

Fitting meta_model on X with all models predictions
OOB_score 0.8668946648426813
konf 11 n_estimators 10 max_depth 6 min_samples_split 4

Fitting meta_model on X with all models predictions
OOB_score 0.8701778385772914
konf 12 n_estimators 10 max_depth 6 min_samples_split 6

Fitting meta_model on X with all models predictions
OOB_score 0.8694938440492476
konf 13 n_estimators 10 min_samples_split 2 min_samples_split 2

Fitting meta_model on X with all models predictions
OOB_score 0.8177838577291382
konf 14 n_estimators 10 min_samples_split 2 min_samples_split 4

Fitting meta_model on X with all models predictions


...

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


OOB_score 0.8779753761969904
konf 63 n_estimators 200 min_samples_split 6 min_samples_split 6

Fitting meta_model on X with all models predictions


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s


OOB_score 0.8775649794801642
{1: 0.9320109439124487, 2: 0.9292749658002736, 3: 0.9264021887824897, 4: 0.8177838577291382, 5: 0.8177838577291382, 6: 0.8177838577291382, 7: 0.8341997264021888, 8: 0.8341997264021888, 9: 0.8341997264021888, 10: 0.8668946648426813, 11: 0.8701778385772914, 12: 0.8694938440492476, 13: 0.8177838577291382, 14: 0.8177838577291382, 15: 0.8177838577291382, 16: 0.8341997264021888, 17: 0.8341997264021888, 18: 0.8341997264021888, 19: 0.8668946648426813, 20: 0.8701778385772914, 21: 0.8694938440492476, 22: 0.9430916552667579, 23: 0.9429548563611491, 24: 0.9399452804377565, 25: 0.8291381668946648, 26: 0.8291381668946648, 27: 0.8291381668946648, 28: 0.8514363885088919, 29: 0.8514363885088919, 30: 0.8514363885088919, 31: 0.8760601915184678, 32: 0.8753761969904241, 33: 0.8763337893296853, 34: 0.8291381668946648, 35: 0.8291381668946648, 36: 0.8291381668946648, 37: 0.8514363885088919, 38: 0.8514363885088919, 39: 0.8514363885088919, 40: 0.8760601915184678, 41: 0.8753761969904

[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.2s finished


In [13]:
# print key where max value in oob_dict
print(max(oob_dict, key=oob_dict.get))  
print('Manuellt hittade jag: n_estimators=100, max_depth=None,min_samples_split=2')


22
Manuellt hittade jag: n_estimators=100, max_depth=None,min_samples_split=2


## Final: learn meta_model med optimala parms och learn allt för modellerna

In [14]:
if True:
    #####    X_stack och y_stack måste vara definierade ovan   #####
    meta_model = learn_meta_model(X_stack, y_stack, save=True,
                                  n_estimators=100,
                                  max_depth=None,
                                  min_samples_split=2,
                                  )



Fitting meta_model on X with all models predictions


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished


OOB_score 0.9430916552667579


In [15]:
if True:
    ### alla värden enligt ovan körningar för modellerna ###
    learn_all_models(totals=True, save=True)
    

train all
[6 460 "'d': 2, 'l2': 9, 'LR': 0.005"]
typ6, Lägg in ant_hästar, 
Number of NaN in cat before: 246
Number of NaN in cat after: 0
best score {'learn': {'Logloss': 0.24209788316629433}}
[1 432 "'d': 4, 'l2': 8, 'LR': 0.005"]
typ1, Lägg in diff motståndare, 
Number of NaN in cat before: 246
Number of NaN in cat after: 0
best score {'learn': {'Logloss': 0.24774235064314085}}
[9 496 "'d': 2, 'l2': 10, 'LR': 0.005"]
typ9, Lägg in ant_hästar, Lägg in diff motståndare, 
Number of NaN in cat before: 246
Number of NaN in cat after: 0
best score {'learn': {'Logloss': 0.2406367497366329}}
[16 496 "'d': 2, 'l2': 10, 'LR': 0.005"]
typ16, Lägg in ant_hästar, Lägg in diff motståndare, 
Number of NaN in cat before: 246
Number of NaN in cat after: 0
best score {'learn': {'Logloss': 0.2406367497366329}}


#### Funktioner för att prioritera mellan hästar

In [ ]:
# se även kelly

In [ ]:
# för en omgång (ett datum) ta ut största diff för streck per avd 
# om only_clear=True, enbart för diff >= 25
def lista_med_favoriter(df_, ant, only_clear):
    df = df_.copy()
    min_diff = 25 if only_clear else 0
    # sortera på avd,streck
    df = df.sort_values(['avd', 'streck'], ascending=[False, False])
    diff_list = []
    for avd in range(1, 8):
        diff = df.loc[df.avd == avd].streck.iloc[0] - \
            df.loc[df.avd == avd].streck.iloc[1]
        if diff >= min_diff:
            diff_list.append((avd, diff))

     # sortera på diff
    diff_list = sorted(diff_list, key=lambda x: x[1], reverse=True)
    return diff_list[:ant]

# temp is a list of tuples (avd, diff). check if avd is in the list
def check_avd(avd, temp):
    for t in temp:
        if t[0] == avd:
            return True
    return False


In [ ]:
def compute_total_insats(df):
    insats = 0
    # group by avd
    summa = df.groupby('avd').avd.count().prod() / 2
    return summa

## Learning-fasen

Gör en scrape på senaste veckan (behövs inte i denna test)

In [ ]:
def model_streck_to_odds(X_):
    X = X_.copy()
    # import modules for linear regression
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_absolute_error as mae
    # import random forest module
    from sklearn.ensemble import RandomForestRegressor

    X_odds = X.loc[X.vodds <= 40]  # remove outliers
    ix_break = int(len(X_odds.datum.unique())*0.75)
    test_start = X_odds.datum.unique()[ix_break]

    X_train, X_test = X_odds[X_odds.datum < test_start], X_odds[X_odds.datum >= test_start]
    y_train, y_test = X_train['vodds'], X_test['vodds']
    X_train = X_train[['streck']].astype(float)
    X_test = X_test[['streck']].astype(float)

    # make a model of RF
    rf = RandomForestRegressor(n_estimators=100, max_depth=6, random_state=0)
    rf.fit(X_train, y_train)
    y_predrf = rf.predict(X_test)
    # make a model and fit it
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    y_predlr = linreg.predict(X_test)

    # print the coefficients
    print('Coefficients:', linreg.coef_)
    # print the mean absolute error
    print("LR Mean absolute error: %.2f" % mae(y_test, y_predlr))
    print("RF Mean absolute error: %.2f" % mae(y_test, y_predrf))

    return linreg, rf


# _, rf = model_streck_to_odds(X)   # used in next cell

# # spara rf
# import pickle
# with open('rf_streck_odds.pkl', 'wb') as f:
#     pickle.dump(rf, f)

Läs in all_data.csv 
Baka ihop senaste vekan med all_data.csv

In [ ]:
def skapa_stack_learning(X_, y, features, iterations=1000, random_state=2022, verbose=False, save=True):
    """
    Skapar en stack med proba och kelly
    X måste ha datum och avd
    """
    X = X_.copy()
    stacked_data = pd.DataFrame()
    
    cbc = CatBoostClassifier(iterations=iterations, loss_function='Logloss', eval_metric='AUC', verbose=verbose)
    for typ in typer:
        nr = typ.name[3:]
        model = typ.learn(X, y, features, iterations=iterations, save=save, verbose=verbose)
        stacked_data['proba'+nr] = typ.predict(X) 
        stacked_data['kelly'+nr] = kelly(stacked_data['proba' + nr], X[['streck']], None)
    
    # print(stacked_data.columns)
    return stacked_data   # enbart stack-info

# fit meta_model
def learn_meta_model(X,y):
    from sklearn.ensemble import RandomForestClassifier

    print('\nFitting meta_model on X with all models predictions')
    
    meta_model = RandomForestClassifier(max_depth=None, n_estimators=100, oob_score=True, verbose=1, n_jobs=10, random_state=2022)
    meta_model.fit(X, y)
    
    print('OOB_score', meta_model.oob_score_)   # 0.9305314451043094
    # pickle save stacking
    pickle.dump(meta_model, open('..\\modeller\\meta.model', 'wb'))
    
    return meta_model


In [ ]:
# read feature list from a file (ej plac)
def read_feature_list(file='../FEATURES.txt'):
    with open(file, 'r') as f:
        return f.read().splitlines()
        

In [ ]:
def liten_cv_timeseries_demo(x_,y_):
    X = x_.copy()
    y = y_.copy()
    print('Holdout validation data from X = X[~validation]')
    from sklearn.model_selection import TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=5)

    for train_index, test_index in tscv.split(X):
        print(f"TRAIN each model on this: {train_index[0]}-{train_index[-1]}, Predict on: {test_index[0]}-{test_index[-1]}")
        print('save the predictions of each model')
        X_train = X.loc[train_index]
        X_test = X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
    print('Train meta_model on all models predictions')
    print('validate meta_model on the validation data')
    
    print('\nHandle stratified and unbalanced data')

Kör learning-skiten här

In [ ]:
FEATURES = read_feature_list("../FEATURES.txt")

X_train, y_train = läs_in_data_och_förbered()
assert X_train.shape[1] == len(FEATURES), f'X_train.shape[1] {X_train.shape[1]} != len(FEATURES) {len(FEATURES)}'
assert set(X_train.columns) == set(FEATURES), f'set(X_train.columns) {set(X_train.columns)} != set(FEATURES) {set(FEATURES)}'
X_train = X_train[FEATURES]  # för att få kolumner i rätt ordning

if True:
    liten_cv_timeseries_demo(X_train, y_train)
else:    
    X_stacked = skapa_stack_learning(X_train, y_train, FEATURES, iterations=100,random_state=2022, verbose=False, save=True)
    # display(X_stacked)
    meta_model = learn_meta_model(X_stacked, y_train)


## Spela-fasen

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import pickle
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import sys

sys.path.append(
    'C:\\Users\peter\\Documents\\MyProjects\\PyProj\\Trav\\spel\\')
import V75_scraping as vs

Scrape-funktion

In [ ]:
def v75_scrape():
    df, strukna = vs.v75_scraping(history=True, resultat=False)
    
    # df = pd.read_csv('../sparad_scrape.csv')
    for f in ['häst','bana', 'kusk', 'h1_kusk', 'h2_kusk', 'h3_kusk', 'h4_kusk', 'h5_kusk', 'h1_bana', 'h2_bana', 'h3_bana', 'h4_bana', 'h5_bana']:
        df[f] = df[f].str.lower()
    return df


In [ ]:
# Alternativ metod
# Ta fram rader för varje typ enligt test-resultaten innan
# låt meta_model välja mellan typerna - hur? Hur maximer insatsen?

Funktion som bygger stack-data från modellerna

In [ ]:
# för stacking ta med alla hästar per typ och proba plus kelly
def build_stack_df(X_):
    X = X_.copy()
    stacked_data = X[['datum','avd', 'startnr','häst']].copy()
    for typ in typer:
        nr = typ.name[3:]
        stacked_data['proba'+nr] = typ.predict(X)
        stacked_data['kelly'+nr] = kelly(stacked_data['proba'+nr], X[['streck']], None)
    return stacked_data


Funktion där meta_model gör predict_proba

In [ ]:
def meta_predict(X_):
    # X_ innehåller även datum,startnr och avd
    extra = ['datum', 'avd', 'startnr', 'häst']
    assert list(X_.columns[:4]) == extra, 'meta_model måste ha datum, avd och startnr, häst för att kunna välja'
    X = X_.copy()
    with open('../modeller\\meta.model', 'rb') as f:
        meta_model = pickle.load(f)
        
    # print(meta_model.predict_proba(X.iloc[:, -8:]))
    X['meta_predict'] = meta_model.predict_proba(X.iloc[:,-8:])[:,1]
    my_columns = extra + list(X.columns)[-9:] 
    
    return X[my_columns]


Funktion som väljer rad

In [ ]:
def comp_cost(antal_rader):
    cost = (antal_rader**2)/2
    return antal_rader,cost

def välj_rad(X_):
    
    max_insats=320
    veckans_rad = X_.copy()
    veckans_rad['välj'] = False

    for avd in veckans_rad.avd.unique():
        max_pred = veckans_rad[veckans_rad.avd == avd]['meta_predict'].max()
        veckans_rad.loc[(veckans_rad.avd == avd) & (veckans_rad.meta_predict == max_pred), 'välj'] = True
    antal_rader=1    
    veckans_rad = veckans_rad.sort_values(by=['meta_predict'], ascending=False)
    
    # 3. Använda ensam favorit för ett par avd? Kolla test-resultat
    # for each row in rad, välj=True if select_func(cost,avd) == True
    cost = antal_rader*0.5
    for i, row in veckans_rad.iterrows():
        new_antal,new_cost = comp_cost(antal_rader+1)
        # print(the_cost)
        if new_cost > max_insats:
            break
        
        antal_rader = new_antal
        cost = new_cost
        veckans_rad.loc[i, 'välj'] = True
        # print(cost)
    veckans_rad.sort_values(by=['välj', 'avd'], ascending=[False, True], inplace=True)

    return veckans_rad


Kör hela välj-rad-skiten här

In [ ]:
X = v75_scrape()
print(X.datum.unique())
df_stack = build_stack_df(X,FEATURES)
df_meta = meta_predict(df_stack)
df_meta.reset_index(drop=True, inplace=True)
veckans_rad = välj_rad(df_meta)
# rename columns 
veckans_rad.rename(columns={'startnr':'nr', 'meta_predict':'Meta', 'välj':'Välj'}, inplace=True)

display(veckans_rad[veckans_rad.välj])
print('kostnad', veckans_rad.välj.sum()**2/2)


# En massa gammal - kanske reusable

Läs in all data

In [ ]:
# only for Learn!
def init_learn():
    df = pd.read_csv('..\\all_data.csv')
    # Följande datum saknar avd==5 och kan inte användas
    saknas = ['2015-08-15', '2016-08-13', '2017-08-12']
    df = df[~df.datum.isin(saknas)]
    X = df.copy()
    X.drop('plac', axis=1, inplace=True)
    # X = ordinal_enc(X, 'häst')
    y = (df.plac == 1)*1   # plac 1 eller 0

    for f in ['häst', 'bana', 'kusk', 'h1_kusk', 'h2_kusk', 'h3_kusk', 'h4_kusk', 'h5_kusk', 'h1_bana', 'h2_bana', 'h3_bana', 'h4_bana', 'h5_bana']:
        X[f] = X[f].str.lower()

    X.reset_index(drop=True, inplace=True)
    y.reset_index(drop=True, inplace=True)

    X, cat_features = tp.prepare_for_catboost(X)
    print('cat_features:', cat_features)
    X.head()
    return X, y, cat_features


modell för streck_to_odds - skall vara fix och inte ändras

In [ ]:
def model_streck_to_odds(X_):
    X = X_.copy()
    # import modules for linear regression
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_absolute_error as mae
    # import random forest module
    from sklearn.ensemble import RandomForestRegressor

    X_odds = X.loc[X.vodds <= 40]  # remove outliers
    ix_break = int(len(X_odds.datum.unique())*0.75)
    test_start = X_odds.datum.unique()[ix_break]

    X_train, X_test = X_odds[X_odds.datum <
                             test_start], X_odds[X_odds.datum >= test_start]
    y_train, y_test = X_train['vodds'], X_test['vodds']
    X_train = X_train[['streck']].astype(float)
    X_test = X_test[['streck']].astype(float)

    # make a model of RF
    rf = RandomForestRegressor(n_estimators=100, max_depth=6, random_state=0)
    rf.fit(X_train, y_train)
    y_predrf = rf.predict(X_test)
    # make a model and fit it
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    y_predlr = linreg.predict(X_test)

    # print the coefficients
    print('Coefficients:', linreg.coef_)
    # print the mean absolute error
    print("LR Mean absolute error: %.2f" % mae(y_test, y_predlr))
    print("RF Mean absolute error: %.2f" % mae(y_test, y_predrf))

    return linreg, rf


linreg, rf = model_streck_to_odds(X)   # used in next cell
# spara rf
import pickle
with open('rf_streck_odds.pkl', 'wb') as f:
    pickle.dump(rf, f)

Engångsgrej för att initiera typ-instanserna med learn

In [ ]:
# Bara första gången. Initierar Typ-klassen
def learn(X_train, y_train, X_test=None, y_test=None, cat_features=[],  iterations=1000,  verbose=False):
    cbc = CatBoostClassifier(iterations=iterations, loss_function='Logloss', eval_metric='AUC', verbose=verbose)
    X_train = remove_features(X_train, remove_mer=['avd','datum'])
    cat_features = X_train.select_dtypes(include=['object']).columns.tolist()
    train_pool = Pool(X_train, label=y_train, cat_features=cat_features)
    if X_test is not None:
        X_test = remove_features(X_test, remove_mer=['avd', 'datum'])
        test_pool = Pool(X_test, label=y_test, cat_features=cat_features)
        cbc.fit(train_pool, eval_set=test_pool, early_stopping_rounds=100, use_best_model=True, verbose=verbose)
    else:
        cbc.fit(train_pool, use_best_model=True, verbose=verbose)
    return cbc

def beräkna_datum(X,fract=0.75):
    ix_break = int(len(X.datum.unique())*fract)
    test_start = X.datum.unique()[ix_break]
    return test_start

if False: # Kör en initiering av typerna 
    Xlearn, cat_features= prepare_for_catboost(X)  
    # print(Xlearn.columns)
    for typ in [typ6, typ1, typ9, typ16]:
        print(typ.name)
        Xtyp = typ.prepare_for_model(Xlearn)                                 ###########

        if not typ.streck:                                                ################
            Xtyp.drop('streck', axis=1, inplace=True)
            
        if True: # använda X_test    
            test_start = beräkna_datum(Xtyp)    
            X_train, X_test = Xtyp[Xtyp.datum < test_start], Xtyp[Xtyp.datum >= test_start]
            y_train, y_test = y[X_train.index], y[X_test.index]
            # print('innan learn',X_train.columns)
            typ_model = learn(X_train, y_train, X_test, y_test)  ##########
            print('best iteration',typ_model.best_iteration_)                             ##########
            print('best score',    typ_model.best_score_)                                 ##########
        # save model
        typ.save_model(typ_model)                                                       ##########                          


Skapa typ6 till typ16

In [ ]:
X,y = läs_in_data_och_förbered()
X,cat_features = tp.prepare_for_catboost(X)
typ6.learn(X,y, iterations=33) # best iter = 25 {'Logloss': 0.23245952928761984, 'AUC': 0.8262112132692319}
typ1.learn(X,y, iterations=39) # best iter = 39 {'Logloss': 0.23278308932319106, 'AUC': 0.826883367187688}
typ9.learn(X,y, iterations=37) # best iter = 37 {'Logloss': 0.23312091900160384, 'AUC': 0.8257515762557716}
typ16.learn(X,y,iterations=37) # best iter = 37 {'Logloss': 0.23312091900160384, 'AUC': 0.8257515762557716}

Skapa stack predict med alla typer

In [ ]:
# stack predict for all models
def stack_predict(X_, models):
    X = X_.copy()
    for typ in typer:
        nr = typ.name[3:]
        X['proba'+nr] = typ.predict(X)
        X['kelly'+nr] = kelly(X['proba'+nr], X[['streck']], None)
    # cols=X.columns[-8]    
    return X


## The complete learning process with all steps in stacking

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()  # The meta model
    
# fit my models on split date for timeseries   
print('START fitting and predicting TimeseriesSplit') 
cross_val_predict=pd.DataFrame()
for id_train, id_test in TimeSeriesSplit(n_splits=5).split(df_stack):  
    for typ in [typ6, typ1, typ9, typ16]:
        typ.learn(df_stack.loc[id_train],y.loc[id_train], iterations=25)
    df_pred = stack_predict(df_stack.loc[id_test], [typ6, typ1, typ9, typ16])
    df_pred['y']=y.loc[id_test]
    cross_val_predict = pd.concat([cross_val_predict, df_pred.iloc[:,-9:]])
       
print('\nFitting my models with all data')
# final fit with all the available data
for typ in [typ6, typ1, typ9, typ16]:
    typ.learn(df_stack, y, iterations=20)

print('\nFitting meta_model on predicted above')
# fit a rf meta_model on cross_val_predict
meta_model = RandomForestClassifier(max_depth=None, n_estimators=100, oob_score=True, verbose=1, n_jobs=10, random_state=2022)
meta_model.fit(cross_val_predict.iloc[:, :-1], cross_val_predict.iloc[:, -1])
print('OOB_score', meta_model.oob_score_)   # 0.9305314451043094
# pickle save stacking
pickle.dump(meta_model, open('..\\modeller\\meta_model.pkl', 'wb'))

In [ ]:
# make prediction on unseen data
def unseen_predictions(X_, models, meta_model):
    X = X_.copy()
    for model in models:
        nr = model.name[3:]
        X['proba'+nr] = model.predict(X)
        X['kelly'+nr] = kelly(X['proba'+nr], X[['streck']], None)
        
    return(meta_model.predict_proba(X.iloc[:, -8:]))

# a small test:
unseen_predictions(df_stack.iloc[-80:,:], [typ6, typ1, typ9, typ16], meta_model)[:,1],y.iloc[-80:].values

